In [ ]:
import sys
import re
import argparse
import pysam
from pyfaidx import Fasta
from Bio.Seq import Seq
from Bio import pairwise2

In [ ]:
## Tmp file input
bam_file = 'tmp/B1_EKDL230008374.merged.bam'
baits_file = 'tmp/Bait_Gateway_AtcDNAlibrary.fas'
araport_file = 'tmp/Araport11_cdna_20220914_representative_gene_model.fas'

In [ ]:
## Defaults
__version__ = '0.1'
DEBUG = 0
mismatches_default = 2
allowed_hd = mismatches_default # tmp
threads_default = 4

## Coordinates on bait sequences
min_length_seq_bait_part = 100
gene_length_default = 9
gene_length = gene_length_default # tmp
accepted_index1 = ['ACT', 'GGA']
accepted_eco571 = ['CTTCAG']
accepted_linker1 = ['ACTAGTTTGATTCGACC']
accepted_index2 = ['CACC']
accepted_linker2 = ['GA']

### Pos 1-9 is the "gene": CATCGACAT, ...
gene_start = 0
gene_end = gene_length
### Pos 10-15 is Eco57I: CTTCAG
eco571_start = 9
eco571_end = 15
### Pos 16-18 is the index: ACT or GGA
index1_start = 15
index1_end = 18
### Pos 19-35 is a linker: ACTAGTTTGATTCGACC
linker1_start = 18
linker1_end = 35
### Pos 36-39 is an index: CACC
index2_start = 35
index2_end = 39
### Pos 40-41 is a linker: GA
linker2_start = 39
linker2_end = 41
### Pos 42-75 is lox66/71: TACCGTTCGTATAATGTATGCTATACGAACGGTA
lox6671_start = 41
lox6671_end = 75
### Pos 76-100 is "after Gateway recombination": TCACAAGTTTGTACAAAAAAGTTGG
agr_start = 75
agr_end = 100

In [ ]:
def hammingDist(seq1, seq2):
    """
    Calculate the Hamming distance in terms of number of changes
    Args: seq1 and seq2 (str)
    Return: distance (int)
    Note: we use .upper() to always compare upper case
    """
    i = 0
    count = 0
    while (i < len(seq1)):
        if (seq1[i].upper() != seq2[i].upper()):
            count += 1
        i += 1
    return count

In [ ]:
## Read bam file. Needs to be indexed: `samtools index -@ 8 B1_EKDL230008374.merged.bam`
bam = pysam.AlignmentFile(bam_file, 'rb', threads=2)

In [ ]:
## Read baits file
baits = Fasta(baits_file, sequence_always_upper=True)

In [ ]:
## Read baits_file and create a hashes with key=SmB-A_AT5G44500.1, value=CATCGACAT ("gene") and value=gene_count
baits_gene_seq_dict = {}
baits_gene_name_dict = {}
baits_gene_count_dict = {}

for seq in baits:
    baits_gene_seq_dict[seq.name] = seq[0:gene_length]
    baits_gene_name_dict[str(seq[0:gene_length])] = seq.name
    baits_gene_count_dict[seq.name] = 0

In [ ]:
## Read araport file
araport = Fasta(araport_file, sequence_always_upper=True)

In [ ]:
## Tmp counters
len_seq_bait_part_ok_counter = 0
len_seq_bait_part_not_ok_counter = 0

In [ ]:
## Test loop
for aso in bam.head(1):
    #help(aso)
    #print("query name ", aso.query_name)
    #print("reference name ", aso.reference_name)
    #print("reference sequence ", aso.get_reference_sequence())
    #print(baits[aso.reference_name][41:75])
    
    ## Araport part
    start_araport_part = aso.query_alignment_end
    seq_araport_part = Seq(str(aso.query_sequence[start_araport_part:]))
    #print(seq_araport_part) # ACTTTATCTCCCAAAACACAAAACAAAAAAAATGGCTTCCACTGCTCTCTCAAGCGCC
    #query_name = aso.query_name.split(':')[0]   # AT1G60950.1
    #print(query_name)
    #print(str(araport[query_name])) # 985 bp sequence
    #seq_araport_db = Seq(str(araport[query_name]))
    #alignments = pairwise2.align.localms(seq_araport_part, seq_araport_db, match=2, mismatch=-1, open=-5, extend=-1, one_alignment_only=True)
    #for match in alignments:
    #    print(match)
    ## Translation
    #trsl = seq_araport_part.translate(table=11, cds=True) # Error
    #fixed_seq = Seq('ACTTTATCTCCCAAAACACAAAACAAAAAAAA-TGGCTTCCACTGCTCTCTCAAGCGCC')
    #fixed_seq.translate(table=11, cds=True)


In [ ]:
## Big loop

## Get aligned sequence object
#for aso in bam.head(100):
for aso in bam.fetch():

    ## Bait part
    seq_bait_part = Seq(str(aso.query_sequence[aso.query_alignment_start:aso.query_alignment_end]))

    ## Discard read if length of seq_bait_part is lower than min_length_seq_bait_part (100)
    if len(seq_bait_part) < min_length_seq_bait_part:
        if DEBUG:
            print(f"DEBUG: SKIPPING: Length of seq_bait_part was lower than min_length_seq_bait_part ({len(seq_bait_part)})")
        len_seq_bait_part_not_ok_counter = len_seq_bait_part_not_ok_counter + 1
        continue

    ## Tmp counter
    len_seq_bait_part_ok_counter = len_seq_bait_part_ok_counter + 1

    ## Go through the parts by position and check if OK
    ## Pos 1-9 is the "gene": CATCGACAT, ...
    gene_seq = seq_bait_part[gene_start:gene_end]
    if baits_gene_name_dict.get(gene_seq) is None:
        if DEBUG:
            print(f"DEBUG: SKIPPING: did NOT find gene {gene_seq} in dict")
        continue
    if DEBUG > 1:
        print(f"DEBUG: found gene in dict: {gene_seq}")

    ## Pos 10-15 is Eco57I: CTTCAG
    if seq_bait_part[eco571_start:eco571_end] not in accepted_eco571:
        if DEBUG:
            print(f"DEBUG: SKIPPING: Eco57I NOT accepted: {seq_bait_part[eco571_start:eco571_end]}")
        continue
    if DEBUG > 1:
        print(f"DEBUG:  found Eco57I: {seq_bait_part[eco571_start:eco571_end]}")

    ## Pos 16-18 is the index1: ACT or GGA
    if seq_bait_part[index1_start:index1_end] not in accepted_index1:
        if DEBUG:
            print(f"DEBUG: SKIPPING: index1 NOT accepted: {seq_bait_part[index1_start:index1_end]}")
        continue
    if DEBUG > 1:
        print(f"DEBUG:   found index1: {seq_bait_part[index1_start:index1_end]}")

    ## Pos 19-35 is a linker1: ACTAGTTTGATTCGACC
    if DEBUG > 1:
        print(f"DEBUG:    found linker1: {seq_bait_part[linker1_start:linker1_end]}")

    ## Pos 36-39 is an index2: CACC
    if DEBUG > 1:
        print(f"DEBUG:     found index2: {seq_bait_part[index2_start:index2_end]}")

    ## Pos 40-41 is a linker2: GA
    if DEBUG > 1:
        print(f"DEBUG:      found linker2: {seq_bait_part[linker2_start:linker2_end]}")

    ## Pos 42-75 is lox66/71: TACCGTTCGTATAATGTATGCTATACGAACGGTA
    read_lox_seq = seq_bait_part[lox6671_start:lox6671_end]
    ref_lox_seq = baits[aso.reference_name][lox6671_start:lox6671_end]
    hd = hammingDist(str(read_lox_seq), str(ref_lox_seq))
    if hd <= allowed_hd:
        if DEBUG > 1:
            print(f"DEBUG:       found lox66/71: {seq_bait_part[lox6671_start:lox6671_end]}")
    else:
        if DEBUG:
            print(f"DEBUG: SKIPPING: HD giher than {allowed_hd} for lox66/71: {seq_bait_part[lox6671_start:lox6671_end]}")
        continue

    ## Pos 76-100 is "after Gateway recombination": TCACAAGTTTGTACAAAAAAGTTGG
    read_agr_seq = seq_bait_part[agr_start:agr_end]
    ref_agr_seq = baits[aso.reference_name][agr_start:agr_end]
    hd = hammingDist(str(read_agr_seq), str(ref_agr_seq))
    if hd <= allowed_hd:
        if DEBUG > 1:
            print(f"DEBUG:        found after Gateway recombination: {seq_bait_part[agr_start:agr_end]}")
    else:
        if DEBUG:
            print(f"DEBUG: SKIPPING: HD giher than {allowed_hd} for after Gateway recombination: {seq_bait_part[agr_start:agr_end]}")
        continue

    ## If All parts of seq_bait_part are OK, parse Araport part
    ## Araport part
    #start_araport_part = aso.query_alignment_end
    #seq_araport_part = Seq(str(aso.query_sequence[start_araport_part:]))

    #query_name = aso.query_name.split(':')[0]   # AT1G60950.1
    #seq_araport_db = Seq(str(araport[query_name]))
    
    ##   Check reading frame for seq_araport_part
    #alignments = pairwise2.align.localms(seq_araport_part, seq_araport_db, match=2, mismatch=-1, open=-5, extend=-1, one_alignment_only=True)
    #for match in alignments:
    #    #print(match)
    #    print(match.seqA)  # seq_araport_part
    #    print(match.start) # Start pos on seq_araport_db
    #    print(match.end)   # End pos on seq_araport_db
    
    ##   Add info on in frame/out of frame

    ## Tmp:  Add to count gene_count
    baits_gene_count_dict[aso.reference_name] = baits_gene_count_dict[aso.reference_name] + 1
    



In [ ]:
# End
bam.close()

print(f"Bam file: {bam_file}")
print(f"Nr bait length OK: {len_seq_bait_part_ok_counter}")
print(f"Nr bait length not OK: {len_seq_bait_part_not_ok_counter}")
print(f"Counts:")

for k,v in sorted(baits_gene_count_dict.items(), key=lambda x:x[1], reverse=True):
    print(k, "=", v)